In [15]:
import csv
import networkx as nx
import matplotlib.pyplot as plt
import random
from math import radians, cos, sin, asin, sqrt
from itertools import combinations
import time
import numpy as np
import pandas as pd

nodes_file_path = '../Sources/map/map_bordeaux_node_list.csv'
edges_file_path = '../Sources/map/map_bordeaux_edge_list.csv'


def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in meters between two points 
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    r = r * 1000
    return c * r

def edges_within_distance_from_id(nodes_pos, origin_id, distance, perturbed_edges):
    """
    Returns a list of edges within a given distance from an origin point identified by its ID,
    and that are contained within the perturbed_edges list.

    Parameters:
    - nodes_pos: Dictionary of node positions with node IDs as keys and (longitude, latitude) as values
    - origin_id: ID of the origin point
    - distance: Distance range in kilometers
    - perturbed_edges: List of tuples representing perturbed edges, where each tuple is (node_id1, node_id2)
    """
    if origin_id not in nodes_pos:
        raise ValueError("Origin ID not found in nodes positions.")
    origin_lon, origin_lat = nodes_pos[origin_id]
    within_distance = []
    for node_id, (lon, lat) in nodes_pos.items():
        if haversine(origin_lon, origin_lat, lon, lat) <= distance:
            within_distance.append(node_id)
    filtered_edges = []
    for edge in perturbed_edges:
        u, v = edge
        if u in within_distance and v in within_distance:    
            filtered_edges.append(edge)
    return filtered_edges

def getNodes(nodes_file_path):
    nodes_list = []
    nodes_pos = {}  # Dictionary for node positions
    with open(nodes_file_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        counter = 0
        for row in csv_reader:
            if counter == 0:
                counter += 1   
            else:
                node_id = str(row[0])
                longitude = float(row[1])
                latitude = float(row[2])
                nodes_list.append(node_id)
                nodes_pos[node_id] = (longitude, latitude)
    return nodes_list, nodes_pos

def setEdges(city, edges_file_path):
    with open(edges_file_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        counter = 0
        for row in csv_reader:
            if counter == 0:
                counter += 1   
            else:
                if row[2] == 'default':
                    row[2] = 50
                city.add_edge(str(row[0]), str(row[1]), weight=float(row[4])/1000/float(row[2])*3600)
    return city   

def add_perturbations(graph, perturbed_edges, random = False):
    """Adds a consistent perturbation to the specified edges in the graph in one direction only.
    
    Parameters:
    - graph: A directed graph object from NetworkX.
    - perturbed_edges: A list of tuples representing the edges to be perturbed.
    """
    for edge in perturbed_edges:
        u, v = edge  # Unpack the edge tuple into start (u) and end (v) nodes
        if graph.has_edge(u, v):  # Check if the edge exists in the specified direction
            if random:
                graph[u][v]['weight'] += random.randint(1, 5) * weight_of_perturbation
            else:
                graph[u][v]['weight'] += weight_of_perturbation
        else:
            print(f"Edge {u} -> {v} not found in graph.")

    return perturbed_edges

def display_graph(graph, nodes_pos, shortest_path, perturbed_edges):
    # Draw the entire graph without labels
    nx.draw(graph, nodes_pos, with_labels=False, node_color='skyblue', node_size=1, edge_color='black', linewidths=1, font_size=15)
    # Draw the shortest path in green
    path_edges = [(shortest_path[n], shortest_path[n+1]) for n in range(len(shortest_path) - 1)]
    nx.draw_networkx_edges(graph, nodes_pos, edgelist=path_edges, edge_color='green', width=2)
    # Draw the perturbed edges in red
    nx.draw_networkx_edges(graph, nodes_pos, edgelist=perturbed_edges, edge_color='red', width=2)
    plt.show()

def display_info(adjusted_path, total_seconds, source, target, type):
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    travel_time = f"{hours}h {minutes}min and {seconds}s"
    print("==================")
    print("Simulation type: " + type)
    print(f"Lenght of the path: {len(adjusted_path)}")
    print(f"Shortest path from {source} to {target}: {' -> '.join(adjusted_path)}")
    print(f"Travel time : {travel_time}")
    print("==================")

def s1(graph, source, target, perturbed_edges, random, display=True):
    """Simulation where perturbations are applied to the weights of the edges of the initial shortest path."""
    # Create a copy of the graph
    G_copy = graph.copy()
    # Find the shortest path before perturbations
    adjusted_path = nx.dijkstra_path(G_copy, source=source, target=target, weight='weight')
    # Apply perturbations
    add_perturbations(G_copy, perturbed_edges, random)
    # Recalculate the length of the initial path with perturbed weights
    perturbed_length = sum(G_copy[u][v]['weight'] for u, v in zip(adjusted_path, adjusted_path[1:]))
    # Convert to total minutes
    total_seconds = perturbed_length
    # Extract hours, minutes, and seconds
    if display:
        display_info(adjusted_path, total_seconds, source, target, "S1")
    return adjusted_path, total_seconds

def s2(graph, source, target, perturbed_edges, random, display=True):
    # Create a copy of the graph
    perturbed_edges_copy = perturbed_edges.copy()
    G_copy = graph.copy()
    G_copy2 = graph.copy()
    # Apply perturbations before finding the path
    add_perturbations(G_copy2, perturbed_edges_copy, random)
    # Find the shortest path
    path = nx.dijkstra_path(G_copy, source=source, target=target, weight='weight')
    # Iteratively adjust the path if a perturbed edge is encountered
    adjusted_path = []
    current_node = source
    while current_node != target:
        next_node = path[path.index(current_node) + 1]
        adjusted_path.append(current_node)
        # Check if any edge from the current node is perturbed
        perturbed = False
        for next_node in G_copy.neighbors(current_node):
            if (current_node, next_node) in perturbed_edges_copy:
                perturbed = True
                # Add the perturbation on the copy
                G_copy[current_node][next_node]['weight'] = G_copy2[current_node][next_node]['weight']
        if(perturbed):
            # Restart djikstra to obtain the new calculated path
            path = nx.dijkstra_path(G_copy, source=current_node, target=target, weight='weight')
        current_node = path[path.index(current_node) + 1]
    adjusted_path.append(current_node)
    adjusted_length = sum(G_copy2[u][v]['weight'] for u, v in zip(adjusted_path[:-1], adjusted_path[1:]))
    # Convert to total minutes
    total_seconds = adjusted_length
    # Extract hours, minutes, and seconds
    if display:
        display_info(adjusted_path, total_seconds, source, target, "S2")
    return adjusted_path, total_seconds

def s3(graph, nodes_pos, distance, source, target, perturbed_edges, random, display=True):
    # Create a copy of the graph
    G_copy = graph.copy()
    G_copy2 = graph.copy()
    perturbed_edges_copy = perturbed_edges.copy()
    # Apply perturbations before finding the path
    add_perturbations(G_copy2, perturbed_edges_copy, random)
    # Iteratively adjust the path if a perturbed edge is encountered
    adjusted_path = [source]
    current_node = source
    path = nx.dijkstra_path(G_copy, source=current_node, target=target, weight='weight')
    path_state = 0
    while current_node != target:
        # Checking for perturbed edges directly on the path
        modified = False
        for node in G_copy.neighbors(current_node):
            if (current_node, node) in perturbed_edges_copy:
                G_copy[current_node][node]['weight'] = G_copy2[current_node][node]['weight']
                modified = True
        # Checking for perturbed edges near the current node
        nearby_perturbed_edges = edges_within_distance_from_id(nodes_pos, current_node, distance, perturbed_edges_copy)
        for edge in nearby_perturbed_edges:
            u, v = edge
            G_copy[u][v]['weight'] = G_copy2[u][v]['weight']
            modified = True

        if modified:
            path = nx.dijkstra_path(G_copy, source=current_node, target=target, weight='weight')
            path_state = 1
        else:
            path_state += 1
            
        adjusted_path.append(path[path_state])
        current_node = path[path_state]
    # Calculation and display of travel time
    adjusted_length = sum(G_copy2[u][v]['weight'] for u, v in zip(adjusted_path[:-1], adjusted_path[1:]))
    # Convert to total minutes
    total_seconds = adjusted_length
    # Extract hours, minutes, and seconds
    if display:
        display_info(adjusted_path, total_seconds, source, target, "S3")
    return adjusted_path, total_seconds

def s4(graph, source, target, perturbed_edges, random, display=True):
    G_copy = graph.copy()
    """Simulation where perturbations are applied before finding the shortest path."""
    # Apply perturbations before finding the path
    add_perturbations(G_copy, perturbed_edges, random)
    # Find the shortest path
    adjusted_path = nx.dijkstra_path(G_copy, source=source, target=target, weight='weight')
    # Display path info with perturbations considered
    perturbed_length = sum(G_copy[u][v]['weight'] for u, v in zip(adjusted_path, adjusted_path[1:]))
    # Convert to total minutes
    total_seconds = perturbed_length
    # Extract hours, minutes, and seconds
    if display:
        display_info(adjusted_path, total_seconds, source, target, "S4")
    return adjusted_path, total_seconds

if __name__ == '__main__':
    
    nodes, nodes_pos = getNodes(nodes_file_path)
    city = nx.DiGraph()
    city.add_nodes_from(nodes)
    city = setEdges(city, edges_file_path)
    all_edges = city.edges(data=False)
    edges_list = list(all_edges)
    edges_number = len(edges_list)
    
    E = []
    L1, L2, L3, L4 = [], [], [], []
    L = []
    l = len(nodes)
    max = l*(l-1)
    print("Nodes number: ", l, " Paths number: ", max)
    
    # Parameters
    random_perturbation = False
    number_of_experiments = 10
    number_of_paths = 10
    weight_of_perturbation = 60
    minimum_length = 80
    for pourcentage in np.arange(0, 5.1, 0.5):
        
        t0 = time.time()
        for k in range(number_of_experiments):
            count = 0
            # create pourcentage pourcent of perturbed edges
            perturbed_edges = random.sample(edges_list, k = int(edges_number * pourcentage / 100))
            print("Starting experiment ", k+1, "/", number_of_experiments)
            while count < number_of_paths:
                try:
                    source = random.choice(nodes)
                    target = random.choice(nodes)

                    path_s1, travel_time_s1 = s1(city, source, target, perturbed_edges, random_perturbation, display=False)
                    #display_graph(city, nodes_pos, path_s1, perturbed_edges)
                    if len(path_s1) >= minimum_length:
                    
                        path_s2, travel_time_s2 = s2(city, source, target, perturbed_edges, random_perturbation, display=False)
                        #display_graph(city, nodes_pos, path_s2, perturbed_edges)
                        
                        path_s3, travel_time_s3 = s3(city, nodes_pos, 200, source, target, perturbed_edges, random_perturbation, display=False)
                        #display_graph(city, nodes_pos, path_s3, perturbed_edges)
                        
                        path_s4, travel_time_s4 = s4(city, source, target, perturbed_edges, random_perturbation, display=False)
                        #display_graph(city, nodes_pos, path_s4, perturbed_edges)

                        L.append([travel_time_s1, 
                                  travel_time_s2, 
                                  travel_time_s3, 
                                  travel_time_s4, 
                                  len(path_s1), 
                                  len(path_s2),
                                  len(path_s3),
                                  len(path_s4),
                                  k, 
                                  pourcentage])
                        
                        if count % 2 == 0:
                            print("Progress: ", pourcentage,"/",5, " ", k+1,"/", number_of_experiments, " ", count+1, "/", number_of_paths)
                        count += 1
                except nx.NetworkXNoPath:
                    # Code à exécuter si aucun chemin n'existe entre i et j
                    pass  # Vous pouvez remplacer "pass" par le code que vous voulez exécuter dans ce cas
        t1 = time.time()
        print(" Time elapsed: ", t1 - t0)                        

Nodes number:  41784  Paths number:  1745860872
Starting experiment  1 / 10
Progress:  0.0 / 5   1 / 10   1 / 10
Progress:  0.0 / 5   1 / 10   3 / 10
Progress:  0.0 / 5   1 / 10   5 / 10
Progress:  0.0 / 5   1 / 10   7 / 10
Progress:  0.0 / 5   1 / 10   9 / 10
Starting experiment  2 / 10
Progress:  0.0 / 5   2 / 10   1 / 10
Progress:  0.0 / 5   2 / 10   3 / 10
Progress:  0.0 / 5   2 / 10   5 / 10
Progress:  0.0 / 5   2 / 10   7 / 10
Progress:  0.0 / 5   2 / 10   9 / 10
Starting experiment  3 / 10
Progress:  0.0 / 5   3 / 10   1 / 10
Progress:  0.0 / 5   3 / 10   3 / 10
Progress:  0.0 / 5   3 / 10   5 / 10
Progress:  0.0 / 5   3 / 10   7 / 10
Progress:  0.0 / 5   3 / 10   9 / 10
Starting experiment  4 / 10
Progress:  0.0 / 5   4 / 10   1 / 10
Progress:  0.0 / 5   4 / 10   3 / 10
Progress:  0.0 / 5   4 / 10   5 / 10
Progress:  0.0 / 5   4 / 10   7 / 10
Progress:  0.0 / 5   4 / 10   9 / 10
Starting experiment  5 / 10
Progress:  0.0 / 5   5 / 10   1 / 10
Progress:  0.0 / 5   5 / 10   3 / 1

In [16]:
#enregistre les liste dans un fichier csv
path = '../Sources/results5.csv'

with open(path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["L1", "L2", "L3", "L4", "len1", "len2", "len3", "len4", "experiment", "perturbation"])
    for row in L:
        writer.writerow(row)
print("Results saved")

df = pd.read_csv(path)
df.describe()

Results saved


,L1,L2,L3,L4,len1,len2,len3,len4,experiment,perturbation
count,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000
mean,837.680320,745.945993,709.290962,662.045704,151.218182,163.572727,160.126364,153.431818,4.500000,2.500000
std,316.231607,261.528921,237.971346,207.575532,50.690174,57.372331,55.584179,50.646466,2.873588,1.581858
min,235.073452,235.073452,235.073452,235.073452,80.000000,66.000000,65.000000,56.000000,0.000000,0.000000
25%,598.983932,549.748220,527.634902,505.787937,111.000000,120.000000,117.000000,114.000000,2.000000,1.000000
50%,790.055455,706.327193,676.685176,641.215821,141.000000,151.000000,149.000000,142.500000,4.500000,2.500000
75%,1006.709157,911.899581,866.106197,805.562381,187.000000,203.250000,198.000000,189.000000,7.000000,4.000000
max,2130.833272,1911.355780,1741.795166,1297.309544,301.000000,424.000000,359.000000,304.000000,9.000000,5.000000
